In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.Plotting import utils as puti
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( puti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)

RgObs={}


In [ ]:




DstInfo = GrU.gridInfo('fv1x1') #,Vgrid=DstVgrid)
lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )

RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src='ERA5', dst='fv1x1' )



Dir_in  = '/glade/campaign/cgd/amp/juliob/ERA5/native_monthly/'
Dir_out = '/glade/campaign/cgd/amp/juliob/ERA5/fv1x1_monthly/'
#######
os.makedirs( Dir_out , exist_ok=True )
        
for y in np.arange( start=2015, stop=2025 ):
    for m in np.arange( start=1, stop=13 ):
        
        
        file_in  = f'ERA5.native.time.{str(y).zfill(4)}-{str(m).zfill(2)}.nc'
        file_out = f'ERA5.fv1x1_zERA.time.{str(y).zfill(4)}-{str(m).zfill(2)}.nc'
        X = xr.open_dataset( f'{Dir_in}/{file_in}' )
        
        print( f'Processing {file_in}')
        
        u=X.U.values
        u_x1 =RgF.Horz(xfld_Src=u , Src='ERA5', Dst='fv1x1' , RegridObj_In= RegridObj )
        print( '   .. U' )
        
        v=X.V.values
        v_x1 =RgF.Horz(xfld_Src=v , Src='ERA5', Dst='fv1x1' , RegridObj_In= RegridObj )
        print( '   .. V' )
        
        q=X.Q.values
        q_x1 =RgF.Horz(xfld_Src=q , Src='ERA5', Dst='fv1x1' , RegridObj_In= RegridObj )
        print( '   .. Q' )
        
        te=X.T.values
        te_x1 =RgF.Horz(xfld_Src=te , Src='ERA5', Dst='fv1x1' , RegridObj_In= RegridObj )
        print( '   .. T' )
        
        og=X.OMEGA.values
        og_x1 =RgF.Horz(xfld_Src=og , Src='ERA5', Dst='fv1x1' , RegridObj_In= RegridObj )
        print( '   .. OMEGA' )
        
        ps=X.PS.values
        ps_x1 =RgF.Horz(xfld_Src=ps , Src='ERA5', Dst='fv1x1' , RegridObj_In= RegridObj )
        print( '   .. PS' )
                
        hyai=X.hyai.values
        hybi=X.hybi.values
        hyam=X.hyam.values
        hybm=X.hybm.values
        
        ilev = ( hyai + hybi )* 1_000. #* 100_000.
        lev  = ( hyam + hybm )* 1_000. #* 100_000.
        
        coords = dict( 
            lon  = ( ["lon"],lon_a ),
            lat  = ( ["lat"],lat_a ),
            lev  = ( ["lev"],lev),
            ilev = ( ["ilev"],ilev),
            time = ( ["time"],  X.time.values ) )
        
        Wds = xr.Dataset( coords=coords  )
        
        Dar = xr.DataArray( data=hyai, dims=('ilev',),
                            attrs=dict( description='interface hybrid eta coordinate A-coeff ',units='1',) ,) 
        Wds['hyai'] = Dar
        
        Dar = xr.DataArray( data=hybi, dims=('ilev',),
                            attrs=dict( description='interface hybrid eta coordinate B-coeff ',units='1',) ,) 
        Wds['hybi'] = Dar
        
        Dar = xr.DataArray( data=hyam, dims=('lev',),
                            attrs=dict( description='mid-level hybrid eta coordinate A-coeff ',units='1',) ,) 
        Wds['hyam'] = Dar
        
        Dar = xr.DataArray( data=hybm, dims=('lev',),
                            attrs=dict( description='mid-level hybrid eta coordinate B-coeff ',units='1',) ,) 
        Wds['hybm'] = Dar
        
        
        
        Dar = xr.DataArray( data=te_x1 , 
                            dims=('time','lev','lat','lon',),
                            attrs=dict( description='Air Temperature',units='K',) ,) 
        Wds['T'] = Dar
        
        Dar = xr.DataArray( data=q_x1 , 
                            dims=('time','lev','lat','lon',),
                            attrs=dict( description='specific humidity',units='kg kg-1',) ,) 
        Wds['Q'] = Dar
        
        Dar = xr.DataArray( data=u_x1 , 
                            dims=('time','lev','lat','lon',),
                            attrs=dict( description='X-wind',units='m s-1',) ,) 
        Wds['U'] = Dar
        
        Dar = xr.DataArray( data=v_x1 , 
                            dims=('time','lev','lat','lon',),
                            attrs=dict( description='Y-wind',units='m s-1',) ,) 
        Wds['V'] = Dar
        
        Dar = xr.DataArray( data=ps_x1 , 
                            dims=('time','lat','lon',),
                            attrs=dict( description='Surface Pressure',units='Pa',) ,) 
        Wds['PS'] = Dar
        
        Wds.to_netcdf( f'{Dir_out}/{file_out}')
        print(f'      Wrote {Dir_out}/{file_out} ' )
